<a href="https://colab.research.google.com/github/s-mostafa-a/lung-cancer-segmentation/blob/master/using_unet3%2B_for_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def _cn_t_d():
  from google.colab import drive
  drive.mount("/content/drive")

In [2]:
def _untar_data():
  !tar -xf ./drive/My\ Drive/Copy_Task06_Lung.tar -C ./

In [3]:
def connect_to_drive():
  !mkdir drive
  _cn_t_d()
  _untar_data()

In [6]:
def install_all_needed_packages():
  !git clone https://github.com/ZJUGiveLab/UNet-Version
  !pip install git+git://github.com/jcreinhold/niftidataset.git
  !mv ./UNet-Version ./un

In [7]:
connect_to_drive()
install_all_needed_packages()

In [8]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('/content/un'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/un']


In [1]:
from typing import Callable, List, Optional

import nibabel as nib
import numpy as np
from PIL import Image
from torch.utils.data.dataset import Dataset
from niftidataset.utils import glob_imgs

class MyCustom3DConverterTo2D(Dataset):
    def __init__(self, source_fns:List[str], target_fns:List[str]):
        from torchvision import transforms as tvt
        from niftidataset.transforms import ToTensor, TrimIntensity, Normalize, AddChannel
        three_d_tfms = tvt.Compose([ToTensor(), AddChannel(), TrimIntensity(-1000.0, 1000.0, 0.0, 255.0), Normalize(is_3d=True)])
        self.source_fns, self.target_fns = source_fns, target_fns
        if len(self.source_fns) != len(self.target_fns) or len(self.source_fns) == 0:
            raise ValueError(f'Number of source and target images must be equal and non-zero')
        self.imgs = [three_d_tfms((nib.load(s).get_data(), nib.load(t).get_data())) for s, t in zip(self.source_fns, self.target_fns)]
        self.target_mean = self.compute_target_mean()
        self.two_d_image = self.convert_images_to_2d()

    @classmethod
    def setup_from_dir(cls, source_dir:str, target_dir:str):
        source_fns, target_fns = glob_imgs(source_dir), glob_imgs(target_dir)
        return cls(source_fns, target_fns)

    def convert_images_to_2d(self):
        from niftidataset.transforms import RandomCrop2D
        list_of_2d_tuples = []
        for src, tgt in self.imgs:
            for i in range(src.shape[0]):
                print(src.shape)
                list_of_2d_tuples.append(RandomCrop2D(src[i], tgt[i]))
        return np.random.shuffle(list_of_2d_tuples)

    def compute_target_mean(self):
        import numpy as np
        means = []
        for src, tgt in self.imgs:
            means.append(tgt.mean())
        return np.asarray(means)

    def get_target_mean(self):
        return self.target_mean

    def __len__(self):
        return len(self.two_d_image)

    def __getitem__(self, idx:int):
        sample = self.two_d_image[idx]
        return sample




def my_custom_train_val_split(source_dir:str, target_dir:str, valid_pct:float=0.2):
    if not (0 < valid_pct < 1):
        raise ValueError(f'valid_pct must be between 0 and 1')
    source_fns, target_fns = glob_imgs(source_dir), glob_imgs(target_dir)
    rand_idx = np.random.permutation(list(range(len(source_fns))))
    cut = int(valid_pct * len(source_fns))
    return (MyCustom3DConverterTo2D(source_fns=[source_fns[i] for i in rand_idx[cut:]],
                                    target_fns=[target_fns[i] for i in rand_idx[cut:]]),
            MyCustom3DConverterTo2D(source_fns=[source_fns[i] for i in rand_idx[:cut]],
                                    target_fns=[target_fns[i] for i in rand_idx[:cut]]))




In [ ]:
np.random.seed(1)
train_dir = '/content/Task06_Lung/'
tds, vds = my_custom_train_val_split(source_dir=train_dir+'imagesTr', target_dir=train_dir+'labelsTr', valid_pct=0.2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
